## MUSIC Generation woohooo

In [1]:
import numpy as np
import tensorflow as tf

import wave, math, struct

from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation

In [2]:
notes_freqs = {
    # Octave 3
    'C3': 130.81, 'C#3': 138.59, 'D3': 146.83, 'D#3': 155.56,
    'E3': 164.81, 'F3': 174.61, 'F#3': 185.00, 'G3': 196.00,
    'G#3': 207.65, 'A3': 220.00, 'A#3': 233.08, 'B3': 246.94,
    
    # Octave 4 (Middle octave)
    'C4': 261.63, 'C#4': 277.18, 'D4': 293.66, 'D#4': 311.13,
    'E4': 329.63, 'F4': 349.23, 'F#4': 369.99, 'G4': 392.00,
    'G#4': 415.30, 'A4': 440.00, 'A#4': 466.16, 'B4': 493.88,
    
    # Octave 5
    'C5': 523.25, 'C#5': 554.37, 'D5': 587.33, 'D#5': 622.25,
    'E5': 659.25, 'F5': 698.46, 'F#5': 739.99, 'G5': 783.99,
    'G#5': 830.61, 'A5': 880.00, 'A#5': 932.33, 'B5': 987.77,
    
    # Octave 6
    'C6': 1046.50, 'C#6': 1108.73, 'D6': 1174.66, 'D#6': 1244.51,
    'E6': 1318.51, 'F6': 1396.91, 'F#6': 1479.98, 'G6': 1567.98,
    'G#6': 1661.22, 'A6': 1760.00, 'A#6': 1864.66, 'B6': 1975.53
}
note_len = len(notes_freqs.keys())

In [3]:
notes = list(notes_freqs.keys())
notes

['C3',
 'C#3',
 'D3',
 'D#3',
 'E3',
 'F3',
 'F#3',
 'G3',
 'G#3',
 'A3',
 'A#3',
 'B3',
 'C4',
 'C#4',
 'D4',
 'D#4',
 'E4',
 'F4',
 'F#4',
 'G4',
 'G#4',
 'A4',
 'A#4',
 'B4',
 'C5',
 'C#5',
 'D5',
 'D#5',
 'E5',
 'F5',
 'F#5',
 'G5',
 'G#5',
 'A5',
 'A#5',
 'B5',
 'C6',
 'C#6',
 'D6',
 'D#6',
 'E6',
 'F6',
 'F#6',
 'G6',
 'G#6',
 'A6',
 'A#6',
 'B6']

In [4]:
note_to_int = {note:i for i, note in enumerate(notes)}
int_to_note = {i:note for i, note in enumerate(notes)}

In [5]:
raw_music_data = [notes[np.random.randint(0,note_len)] for i in range(8000)] # random music data

Data preparation

In [6]:
sequence_length = 12
network_input = []
network_output = []

for i in range(len(raw_music_data)-sequence_length):
    seq_in = raw_music_data[i: i+sequence_length]
    seq_out = raw_music_data[i+sequence_length]
    network_input.append([note_to_int[char] for char in seq_in])
    network_output.append(note_to_int[seq_out])

    print(seq_in, '--->', seq_out)

['E5', 'D4', 'E5', 'C6', 'F5', 'D4', 'G#4', 'F6', 'G4', 'A#3', 'E5', 'E6'] ---> F5
['D4', 'E5', 'C6', 'F5', 'D4', 'G#4', 'F6', 'G4', 'A#3', 'E5', 'E6', 'F5'] ---> F6
['E5', 'C6', 'F5', 'D4', 'G#4', 'F6', 'G4', 'A#3', 'E5', 'E6', 'F5', 'F6'] ---> C6
['C6', 'F5', 'D4', 'G#4', 'F6', 'G4', 'A#3', 'E5', 'E6', 'F5', 'F6', 'C6'] ---> F4
['F5', 'D4', 'G#4', 'F6', 'G4', 'A#3', 'E5', 'E6', 'F5', 'F6', 'C6', 'F4'] ---> F4
['D4', 'G#4', 'F6', 'G4', 'A#3', 'E5', 'E6', 'F5', 'F6', 'C6', 'F4', 'F4'] ---> A3
['G#4', 'F6', 'G4', 'A#3', 'E5', 'E6', 'F5', 'F6', 'C6', 'F4', 'F4', 'A3'] ---> E5
['F6', 'G4', 'A#3', 'E5', 'E6', 'F5', 'F6', 'C6', 'F4', 'F4', 'A3', 'E5'] ---> C#5
['G4', 'A#3', 'E5', 'E6', 'F5', 'F6', 'C6', 'F4', 'F4', 'A3', 'E5', 'C#5'] ---> D#5
['A#3', 'E5', 'E6', 'F5', 'F6', 'C6', 'F4', 'F4', 'A3', 'E5', 'C#5', 'D#5'] ---> G#4
['E5', 'E6', 'F5', 'F6', 'C6', 'F4', 'F4', 'A3', 'E5', 'C#5', 'D#5', 'G#4'] ---> A#4
['E6', 'F5', 'F6', 'C6', 'F4', 'F4', 'A3', 'E5', 'C#5', 'D#5', 'G#4', 'A#4'] ---> 

In [7]:
n_patterns = len(network_input)

In [8]:
x = np.reshape(network_input, (n_patterns, sequence_length, 1))
x

array([[[28],
        [14],
        [28],
        ...,
        [10],
        [28],
        [40]],

       [[14],
        [28],
        [36],
        ...,
        [28],
        [40],
        [29]],

       [[28],
        [36],
        [29],
        ...,
        [40],
        [29],
        [41]],

       ...,

       [[12],
        [18],
        [33],
        ...,
        [30],
        [31],
        [12]],

       [[18],
        [33],
        [22],
        ...,
        [31],
        [12],
        [32]],

       [[33],
        [22],
        [ 4],
        ...,
        [12],
        [32],
        [41]]], shape=(7988, 12, 1))

In [9]:
from keras.utils import to_categorical

y = to_categorical(network_output)
y.shape

(7988, 48)

Build the model

In [10]:
model = Sequential()
model.add(LSTM(256, input_shape=(x.shape[1], x.shape[2])))
model.add(Dense(1000, activation='relu'))
model.add(Dense(note_len, activation='softmax'))

d:\DAI Aug-25\softwares\conda_env\dnn\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 256)            │       264,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1000)           │       257,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 48)             │        48,048 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 569,240 (2.17 MB)

 Trainable params: 569,240 (2.17 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(x,y, epochs=10)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.0238 - loss: 3.8838
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.0207 - loss: 3.8722
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.0244 - loss: 3.8704
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.0219 - loss: 3.8695
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.0263 - loss: 3.8682
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.0273 - loss: 3.8665
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.0259 - loss: 3.8634
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.0275 - loss: 3.8608
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.0288 - loss: 3.8587
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.0295 - loss: 3.8541


### Generating new melody sequence

In [14]:
start_index = np.random.randint(0, len(network_input))
len(network_input)
start_index

7613

In [15]:
pattern = network_input[start_index]
pattern

[0, 17, 17, 31, 36, 12, 20, 38, 28, 7, 17, 37]

In [16]:
generated_melody = []
for i in range(32):
    x_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction = model.predict(x_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    generated_melody.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

Saving as audio file

In [19]:
with wave.open('my_music2.wav', 'w') as wav_file:
    # channels (mono = 1), byte size, sample rate, 0, 'None', (compressed or not)
    wav_file.setparams((1, 2, 44100, 0, 'NONE', 'not compressed'))
    for note in generated_melody:
        freq = notes_freqs[note]
        num_samples = int(0.5*44100) # duration * sample rate
        for i in range(num_samples):
            # sample rate
            t = float(i) / 44100
            value = int(32767 * 0.5 * math.sin(2*math.pi*freq*t))
            # store in binary
            data = struct.pack('<h', value) # h for header
            wav_file.writeframes(data) 